
# Zero/Few-shot In-context Learning 실습 노트북
**작성일**: 2025-07-18 by Yeongtak Oh (Credit : DSAIL LAb, SNU)

**목표**:  
- Zero-shot 및 Few-shot 학습 방식 이해  
- Chain-of-Thought 및 Tool-use 프롬프트 실습  
- 사용자 입력 기반 인터랙션 실습  
- 결과 CSV 저장, 체크리스트, 퀴즈 포함

---
> 본 노트북은 HD현대 실습을 위해 교육용 자료로서 준비되었으며, PyTorch와 HuggingFace Transformers 라이브러리를 사용합니다.


In [ ]:
!pip install transformers datasets ipywidgets --quiet


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

 # https://huggingface.co/gpt2-medium
model_name = "gpt2-medium" # "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.eval()

log = []


In [ ]:
def generate_completion(prompt, max_new_tokens=50):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id
        )
    output = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)
    log.append({"prompt": prompt, "response": output})
    return output



## 실습 체크리스트

- 1) Zero-shot 번역 실습 실행  
- 2) Few-shot 번역 실습 실행  
- 3) 감정 분석 예제 실행  
- 4) 사용자 직접 프롬프트 실습  
- 5) Chain-of-Thought 예제 실행  
- 6) Tool-use 예제 실행  
- 결과를 CSV로 저장


In [ ]:
# zero-shot은 demonstration x
print("Zero-shot 예제")
prompt = "Translate English to French: I like pizza."
print("Prompt:", prompt)
print("Generated:", generate_completion(prompt))

# few-shot은 demonstration o
print("\n Few-shot 예제")
few_shot = (
    "Translate English to French:\n"
    "English: Hello.\nFrench: Bonjour.\n"
    "English: Good night.\nFrench: Bonne nuit.\n"
    "English: I like pizza.\nFrench:"
)
print("Prompt:\n", few_shot)
print("Generated:", generate_completion(few_shot))


In [ ]:
# example 1: sentimental analysis
print("\n 감정 분석 Few-shot")
emotion_prompt = (
    "Tweet: I hate everything.\nSentiment: Negative\n"
    "Tweet: I love sunshine.\nSentiment: Positive\n"
    "Tweet: I want to cry.\nSentiment:"
)
print(generate_completion(emotion_prompt))

# CoT example
print("\n Chain-of-Thought 예제")
cot = (
    "Q: If a train has 5 cars and each car has 20 people, how many total people?\n"
    "A: Let's think step by step. Each car has 20 people. There are 5 cars. So 5 * 20 = 100. Answer: 100"
    "\n\nQ: There are 3 boxes. Each box has 4 apples. How many apples?\nA:"
)
print(generate_completion(cot))

# Tool use example
print("\n Tool-use 예제")
tool_prompt = (
    "Question: What is 15 * 12?\n"
    "Let's use a calculator: 15 * 12 = 180.\nAnswer: 180\n"
    "Question: What is 9 * 11?\nLet's use a calculator:"
)
print(generate_completion(tool_prompt))


In [ ]:
# With on your own prompts (PLAYGROUND)
prompt_input = widgets.Textarea(
    value='Input your prompt here...',
    placeholder='Enter any instruction or query',
    description='Prompt:',
    layout=widgets.Layout(width='100%', height='100px')
)
button = widgets.Button(description='Generate')
output = widgets.Output()

@output.capture()
def on_click(b):
    output.clear_output()
    prompt = prompt_input.value
    result = generate_completion(prompt)
    print("Generated:", result)

button.on_click(on_click)
display(prompt_input, button, output)


In [ ]:
# Output save files
df = pd.DataFrame(log)
df.to_csv("incontext_outputs.csv", index=False)
print("💾 Saved as incontext_outputs.csv")



## 📝 퀴즈 (5분)

1. Zero-shot learning과 Few-shot learning의 차이를 서술하시오.  
2. Chain-of-Thought prompting이 모델 성능 향상에 어떤 영향을 미치는가?  
3. Tool-use prompting의 한계점은 무엇인가?

> 답안은 아래 셀에 작성하세요.


In [ ]:
# 여기에 작성하세요.
